In [1]:
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GeneralizedLinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType, DateType
import pyspark.sql.functions as f
import os
from custom_utils import project_base_dir, fuel_type, rolling_window_size, visualisation_dir
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.\
    builder.\
    appName("ml_training-notebook").\
    getOrCreate()

23/10/21 14:24:13 WARN Utils: Your hostname, DIC resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/10/21 14:24:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/21 14:24:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
training_schema = StructType() \
      .add("station_uuid",StringType(),True) \
      .add("date",DateType(),True) \
      .add("hour",IntegerType(),True) \
      .add("weekday",IntegerType(),True) \
      .add("deviation",DoubleType(),True) \
      .add("cloudcover",IntegerType(),True) \
      .add("rain",DoubleType(),True) \
      .add("temperature_2m",DoubleType(),True) \
      .add("hour_sin",DoubleType(),True) \
      .add("hour_cos",DoubleType(),True) \
      .add("weekday_sin",DoubleType(),True) \
      .add("weekday_cos",DoubleType(),True)

In [4]:
training_dataframe = spark.read.format("csv") \
      .option("header", True) \
      .schema(training_schema) \
      .load(os.path.join(project_base_dir, "outputs/training_data.csv"))

In [5]:
input_columns = ["cloudcover", "rain", "temperature_2m", "hour_sin", "hour_cos", "weekday_sin", "weekday_cos"]
assembler = VectorAssembler(
    inputCols=input_columns,
    outputCol="features")

data = assembler.transform(training_dataframe)
final_data = data.select("features", "deviation")

train_data, test_data = final_data.randomSplit([0.9, 0.1], seed=42)

In [6]:
def evaluate_predictions(predictions):
    evaluator = RegressionEvaluator(labelCol="deviation", predictionCol="predicted_deviation", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))

    evaluator_r2 = RegressionEvaluator(labelCol="deviation", predictionCol="predicted_deviation", metricName="r2")
    r2 = evaluator_r2.evaluate(predictions)
    print("R-squared (R2) on test data: {:.3f}".format(r2))

In [7]:
def train_and_evaluate_model(model, training_data, test_data):
    l_model = model.fit(training_data)
    predictions = l_model.transform(test_data)
    evaluate_predictions(predictions)
    return l_model

In [8]:
def show_lr_feature_importance(feature_importances, input_columns):
    feature_importance = sorted(list(zip(input_columns, map(abs, feature_importances), feature_importances)), key=lambda x: x[1], reverse=True)

    print("Feature Importance:")
    for feature, importance, coef in feature_importance:
        print("  {}: {:.5f}".format(feature, coef))

In [9]:
def plot_residuals(predictions, model_name):
    residuals_histogram = predictions \
        .withColumn("residuals", f.col("predicted_deviation") - f.col("deviation")) \
        .select('residuals') \
        .rdd.flatMap(lambda x: x) \
        .histogram(50)
    
    ax = pd.DataFrame(
        list(zip(*residuals_histogram)), 
        columns=['bin', 'frequency']
    ).set_index(
        'bin'
    ).plot(
        kind='bar',
        figsize=(10, 5),
        title=f"Distribution of residuals with {model_name}"
    )
    labels = [item.get_text() for item in ax.get_xticklabels()]
    tick_labels = [float(label) for label in labels]
    ax.set_xticklabels([f"{label:.03f}" for label in tick_labels])
    ax.set_xlabel(f"Error between predicted and actual deviation of the {fuel_type} price from its {rolling_window_size}-day rolling average")
    fig = ax.get_figure()
    fig.tight_layout()
    fig.savefig(os.path.join(visualisation_dir, f"{model_name}_prediction_residuals.svg"))
    plt.close(fig)

### Linear regressor

In [10]:
lr = LinearRegression(featuresCol="features", labelCol="deviation", predictionCol="predicted_deviation")

In [11]:
lr_model = train_and_evaluate_model(lr, train_data, test_data)

23/10/21 14:24:18 WARN Instrumentation: [b0d7a8df] regParam is zero, which might cause numerical instability and overfitting.
23/10/21 14:24:20 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/10/21 14:24:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Root Mean Squared Error (RMSE) on test data: 0.047
R-squared (R2) on test data: 0.345


In [12]:
show_lr_feature_importance(lr_model.coefficients, input_columns)

Feature Importance:
  hour_sin: 0.04546
  hour_cos: -0.00571
  weekday_sin: -0.00055
  rain: -0.00034
  weekday_cos: -0.00029
  temperature_2m: 0.00010
  cloudcover: 0.00002


In [13]:
plot_residuals(lr_model.transform(test_data), LinearRegression.__name__)

### Decision tree regressor

In [14]:
dtr = DecisionTreeRegressor(featuresCol="features", labelCol="deviation", predictionCol="predicted_deviation")

In [15]:
dtr_model = train_and_evaluate_model(dtr, train_data, test_data)

Root Mean Squared Error (RMSE) on test data: 0.040
R-squared (R2) on test data: 0.526


In [16]:
show_lr_feature_importance(dtr_model.featureImportances.toArray(), input_columns)

Feature Importance:
  hour_sin: 0.76005
  temperature_2m: 0.13586
  hour_cos: 0.10088
  cloudcover: 0.00320
  rain: 0.00000
  weekday_sin: 0.00000
  weekday_cos: 0.00000


In [17]:
plot_residuals(dtr_model.transform(test_data), DecisionTreeRegressor.__name__)

### Random forest regressor

In [18]:
rfr = RandomForestRegressor(featuresCol="features", labelCol="deviation", predictionCol="predicted_deviation")

In [19]:
rfr_model = train_and_evaluate_model(rfr, train_data, test_data)

Root Mean Squared Error (RMSE) on test data: 0.040
R-squared (R2) on test data: 0.504


In [20]:
show_lr_feature_importance(rfr_model.featureImportances.toArray(), input_columns)

Feature Importance:
  hour_sin: 0.70380
  hour_cos: 0.18665
  temperature_2m: 0.09888
  cloudcover: 0.00685
  weekday_sin: 0.00171
  rain: 0.00161
  weekday_cos: 0.00050


In [21]:
plot_residuals(rfr_model.transform(test_data), RandomForestRegressor.__name__)

### Generalised linear regression

In [22]:
glm = GeneralizedLinearRegression(featuresCol="features",
                                  labelCol="deviation",
                                  predictionCol="predicted_deviation",
                                  family="gaussian",
                                  link="identity",
                                  maxIter=10,
                                  regParam=0.3)

In [23]:
glm_model = train_and_evaluate_model(glm, train_data, test_data)

Root Mean Squared Error (RMSE) on test data: 0.054
R-squared (R2) on test data: 0.109


In [24]:
show_lr_feature_importance(glm_model.coefficients, input_columns)

Feature Importance:
  hour_sin: 0.00728
  hour_cos: -0.00300
  rain: 0.00024
  weekday_sin: -0.00007
  temperature_2m: -0.00003
  weekday_cos: -0.00002
  cloudcover: 0.00001


In [25]:
plot_residuals(glm_model.transform(test_data), GeneralizedLinearRegression.__name__)